<a href="https://colab.research.google.com/github/mkane2/baptisms/blob/main/network_timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Sample csv](https://github.com/mkane2/baptisms/blob/main/DRC%20Schoharie/drc-schoharie.csv)

In [ ]:
!pip install gender_guesser

In [ ]:
!pip install python-louvain
import community.community_louvain as community_louvain

In [ ]:
import csv, urllib.request
import json
from operator import itemgetter 
from itertools import combinations

from networkx.readwrite import json_graph 
import networkx as nx 
from networkx.algorithms import community 
from networkx.algorithms.approximation import average_clustering

import matplotlib.pyplot as plt
import pandas as pd
from datetime import date, timedelta, datetime
# from dateutil.relativedelta import relativedelta
import numpy as np

import gender_guesser.detector as gender
import os
from os.path import exists

from google.colab import files
import shutil

In [ ]:
# remote_files = [
#                 {"congregation": "gravesend-drc",
#                  "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/gravesend-drc.csv",
#                  "denomination": "dutch reformed",
                  #  "latitude": 40.598,
                  #  "longitude": -73.971}
#                 ]
remote_files = [
                  {"congregation": "gravesend-drc",
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/gravesend-drc.csv",
                 "denomination": "dutch reformed",
                  "latitude": 40.598,
                  "longitude": -73.971},
                  {"congregation": "flatlands-drc",
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/flatlands-drc.csv",
                 "denomination": "dutch reformed",
                  "latitude":40.621,
                  "longitude":-73.935},
                {"congregation": "drc-schoharie", 
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/drc-schoharie.csv",
                 "denomination": "dutch reformed",
                "latitude": 42.6623856,
                "longitude": -74.3134161},
                {"congregation": "fthunter-anglican", 
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/fthunter-anglican.csv",
                 "denomination": "anglican",
                "latitude": 42.939625,
                "longitude": -74.286283},
                {"congregation": "detroit-catholic", 
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/detroit-catholic.csv",
                 "denomination": "catholic",
                "latitude": 42.283333,
                "longitude": -83},
                {"congregation": "huguenot-nyc", 
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/huegenot-nyc.csv",
                 "denomination": "huguenot",
                "latitude": 40.7634666,
                "longitude": -73.9687545},
                {"congregation": "albany-lutheran", 
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/albany-lutheran.csv",
                 "denomination": "lutheran",
                "latitude": 42.6619892,
                "longitude":-73.7776913},
                {"congregation": "rensl-lutheran", 
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/rensl-lutheran.csv",
                 "denomination": "lutheran",
                "latitude": 42.7612678,
                "longitude": -73.5955521},
                {"congregation": "schoharie-lutheran", 
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/schoharie-lutheran.csv",
                 "denomination": "lutheran",
                "latitude": 42.664444,
                "longitude": -74.3125},
                {"congregation": "drc-albany", 
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/drc-albany.csv",
                 "denomination": "dutch reformed",
                "latitude": 42.653611,
                "longitude": -73.750556},
                {"congregation": "sleepyhollow-drc", 
                "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/sleepyhollow-drc.csv",
                 "denomination": "dutch reformed",
                "latitude": 41.090278,
                "longitude": -73.861667},
                  {"congregation": "flatbush-drc", 
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/flatbush-drc.csv",
                 "denomination": "dutch reformed",
                "latitude": 40.65,
                "longitude": -73.959167},
                {"congregation": "bergen-drc",
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/bergen-drc.csv",
                 "denomination": "dutch reformed",
                "latitude": 40.726944,
                "longitude": -74.069444},
                {"congregation": "westcamp-lutheran",
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/westcamp-lutheran.csv",
                 "denomination": "lutheran",
                "latitude": 41.541667,
                "longitude": -74.063056},
                {"congregation": "caughnawaga-drc",
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/caughnawaga-drc.csv",
                 "denomination": "dutch reformed",
                "latitude": 42.954444,
                "longitude": -74.375556},
                {"congregation": "schenectady-drc",
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/schenectady-drc.csv",
                 "denomination": "dutch reformed",
                "latitude":42.814167,
                "longitude":-73.937222},
                {"congregation": "deerpark-drc",
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/deerpark-drc.csv",
                 "denomination": "dutch reformed",
                "latitude":41.45,
                "longitude":-74.65},
                {"congregation": "helderburgh-drc",
                 "filename": "https://raw.githubusercontent.com/mkane2/baptisms/main/edge_lists/helderburgh-drc.csv",
                 "denomination": "dutch reformed",
                "latitude":42.701389,
                "longitude":-73.925278}
                ]
statsdfs = []
degdfs = []

In [ ]:
female_names = pd.read_csv("https://raw.githubusercontent.com/mkane2/baptisms/main/female_names.csv")
male_names = pd.read_csv("https://raw.githubusercontent.com/mkane2/baptisms/main/male_names.csv")
gc = gender.Detector()

In [ ]:
def setup(c, f, d, lt, ln):
  global congregation
  global denomination
  global latitude
  global longitude
  global filename
  global df
  global start_date
  global end_date
  congregation = c
  filename = f
  denomination = d
  latitude = lt
  longitude = ln
  df = pd.read_csv(filename)
  #FINDING MAX AND MIN
  start_date = df['year'].min()
  end_date = df['year'].max()
  os.mkdir('./' + congregation + '/')

In [ ]:
def getedges(): 
  global edges
  response = urllib.request.urlopen(filename)
  lines = [l.decode('utf-8') for l in response.readlines()]
  cr = csv.reader(lines)
  edges = [tuple(e) for e in cr][1:]

## Make the initial graph

In [ ]:
def makegraph():
  global G
  G = nx.Graph()
  for e in edges:
    calc_end = int(e[0]) + 20
    if calc_end > end_date:
      calc_end = end_date
    G.add_edge(e[1], e[2], start=int(e[0]), end=calc_end)

  G = nx.relabel.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute="name")
  G.remove_edges_from(nx.selfloop_edges(G))

## Extract the giant component

In [ ]:
def make_giant():  
  global L
  global connected_subgraphs
  # find the number of subgraphs in the overall network
  connected_subgraphs = [G.subgraph(cc) for cc in nx.connected_components(G)]
  print("Number of subgraphs: " + str(len(connected_subgraphs)))

  # get the giant component of the overall network
  Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
  L = G.subgraph(Gcc[0])
  K = G.subgraph(Gcc[1])
  print(G)
  print(L)
  print(K)

  global giant_percent_edges
  global giant_percent_nodes
  giant_percent_nodes = L.number_of_nodes()/G.number_of_nodes()
  giant_percent_edges = L.number_of_edges()/G.number_of_edges()
  print(giant_percent_nodes, giant_percent_edges)

In [ ]:
# L.nodes[0]

## Get degree sequence

In [ ]:
def get_degseq():
  global degrees
  degrees = [val for (node, val) in sorted(L.degree(), key=lambda x: x[-1])]
# print(degrees)

## Get triads

In [ ]:
def get_triads():
  global triad_class
  triad_class = {}
  for nodes in combinations(L.nodes, 3):
      n_edges = L.subgraph(nodes).number_of_edges()
      triad_class.setdefault(n_edges, []).append(nodes)


In [ ]:
# print("unconnected", len(triad_class[0]))
# print("dyad", len(triad_class[1]))
# print("incomplete triad", len(triad_class[2]))
# print("complete triad", len(triad_class[3]))

## Extract nodes

In [ ]:
def get_nodes():
  global nodedf
  rows = list(G.nodes(data=True))
  listofdicts = []
  for n in rows:
    thisdict = {"id": n[0], "name": n[1]['name']}
    listofdicts.append(thisdict)
  nodedf = pd.DataFrame(listofdicts) 
  nodedf.to_csv(congregation + '/' + congregation + '_nodelist.csv', index=False) 

## Assign gender

In [ ]:
def assign_gender():
  global nodegender
  data = congregation + '/' + congregation + '_nodelist.csv'
  gender_out = congregation + '/' + congregation + '_nodelist_gender.csv'

  record = 1
  with open(data, 'r') as file:
    for line in file:
      if record == 1:
        newline = line.strip().split(',')
        stuff = ['gender']
        newline.extend(stuff)
        with open(gender_out, 'a') as outputfile:
          outputwriter = csv.writer(outputfile)
          outputwriter.writerow(newline)
      else:
        # print(line)
        name = line.split(',')[1]
        first = name.split(' ')[0]
        get_this = gc.get_gender(first)
        # gender = get_this['gender']
        # prob = get_this['probability']
        newline = line.strip().split(',')
        newline.extend([get_this])
        with open(gender_out, 'a') as outputfile:
            outputwriter = csv.writer(outputfile)
            outputwriter.writerow(newline)
      record += 1

  nodegender = pd.read_csv(gender_out)
  nodegender['gender'].fillna('unknown', inplace=True)
  nodegender.loc[nodegender['gender'] == 'andy', 'gender'] = "unknown"
  nodegender.loc[nodegender['gender'] == 'mostly_female', 'gender'] = "female"
  nodegender.loc[nodegender['gender'] == 'mostly_male', 'gender'] = "male"

In [ ]:
# nodegender.gender.value_counts()

In [ ]:
# nodegender.loc[nodegender['gender'] == "unknown"]

In [ ]:
def gender_unknown():
  for index, row in nodegender.loc[nodegender['gender'] == "unknown"].iterrows():
      first = row['name'].split(' ')[0]
      if (first in female_names['name'].values):
        nodegender.loc[nodegender['name'] == row['name'], 'gender'] = "female"
      if (first in male_names['name'].values):
        nodegender.loc[nodegender['name'] == row['name'], 'gender'] = "male"

In [ ]:
# nodegender.loc[nodegender['gender'] == "unknown"]

In [ ]:
def set_gender():
  node_attr = nodegender.set_index('id').to_dict('index')
  nx.set_node_attributes(L, node_attr)

In [ ]:
# L.nodes[0]

In [ ]:
def gender_assort(graph):
  global gender_assortivity
  gender_assortivity = nx.attribute_assortativity_coefficient(graph, attribute='gender')
  # print(gender_assortivity)

## Assign race

In [ ]:
def assign_race():
  print("assign race")
  nodegender['race'] = ""
  nodegender.loc[nodegender['name'].str.contains("Indian"), 'race'] = "Native"
  nodegender.loc[nodegender['name'].str.contains("Negro"), 'race'] = "Black"
  nodegender.loc[nodegender['name'].str.contains("Slave"), 'race'] = "Black"
  nodegender.loc[nodegender['name'].str.contains("slave"), 'race'] = "Black"
  nodegender.loc[nodegender['race'] == '', 'race'] = "white"
  race_out = congregation + '/' + congregation + "_nodelist_gender_race.csv"
  nodegender.to_csv(race_out,index=False)
  # nodegender

In [ ]:
# nodegender.race.value_counts()

In [ ]:
def set_race():
  print("set race")
  noderace = nodegender.drop(columns=["gender"])
  raceattr = noderace.set_index('id').to_dict('index')
  # noderace
  nx.set_node_attributes(L, raceattr)

## Find bridges

In [ ]:
def set_bridges(graph):
  print("set bridges")
  global male_bridges
  global female_bridges
  global bridges
  bridgeattr = "false"
  nx.set_node_attributes(graph, bridgeattr, "bridge")
  bridges = list(nx.articulation_points(graph))

  for b in bridges:
    if graph.nodes[b]['gender'] == 'female':
      nx.set_node_attributes(graph, {b:"female_bridge"}, 'bridge')
    else: 
      nx.set_node_attributes(graph, {b:"male_bridge"}, 'bridge')
  male_bridges = len([x for x,y in graph.nodes(data=True) if y['bridge']=="male_bridge"])
  female_bridges = len([x for x,y in graph.nodes(data=True) if y['bridge']=="female_bridge"])
  print("male bridges", male_bridges, "female bridges", female_bridges)

## Build the list of dates

In [ ]:
def set_degree():
  degree_dict = dict(L.degree(L.nodes()))
  nx.set_node_attributes(L, degree_dict, "degree")
  betweenness_dict = nx.betweenness_centrality(L)
  nx.set_node_attributes(L, betweenness_dict, "betweenness")

In [ ]:
def set_end():
  for n in L.nodes():
    cnxedges = L.adj[n]
    thislist = []
    for nbr, datadict in G.adj[n].items():
      thislist.append(int(datadict['end']))
    sortedlist = sorted(thislist, reverse=True)
    nx.set_node_attributes(L, {n: sortedlist[0]}, 'end')

In [ ]:
def make_years():
  global listofrows
  global start_date
  preferential_edges = []
  jacc_edges = []
  listofrows = []
  M = None
  year_count = 1
  # while the start date is less than the end date
  while start_date <= (end_date + 1):
    # select only the edges that were created 20 years before the start date, up to the start date
    selected_edges = [(u,v) for u,v,e in L.edges(data=True) if int(e['start']) in range(start_date - 20, start_date + 1)]
    # print(selected_edges)
    # make a new graph with the time filtered edges
    H = nx.Graph(selected_edges)
    print(H)
    # print("L", L.nodes.data())
    # print("H", H.nodes.data())
    # print("H edges", list(H.edges(data=True)))
    # print("H nodes", list(H.nodes(data=True)))
    for node in H.nodes():
      # print(L.nodes[node])
      attrs = L.nodes[node]
      H.nodes[node].update(attrs) 
      # nx.set_node_attributes(H, attrs)
      # print(H.nodes[node])
    if H.number_of_nodes() == 0:
      start_date += 1
      year_count += 1
      continue

    # make a dictionary for the filtered network's degree
    degree_dict = dict(H.degree(H.nodes()))

    # write the start date and degree to the overall network
    f_date = start_date
    degree_time = 'degree_' + str(f_date)
    nx.set_node_attributes(L, 0, degree_time)
    nx.set_node_attributes(L, degree_dict, degree_time)
    
    # write the betweenness to the overall network
    betweenness_dict = nx.betweenness_centrality(H)
    betweenness_time = 'between_' + str(f_date)
    nx.set_node_attributes(L, 0, betweenness_time)
    nx.set_node_attributes(L, betweenness_dict, betweenness_time)

    start_dict = dict()
    end_dict = dict()
    # M is the previous year graph
    # check if a node exists in the previous year M
    # if it does NOT exist in the previous year, write the f_date into the start_dict
    # if it does exist in the previous year and the current year, don't write anything
    # if it exists in the previous year but NOT the current year, write the f_date into the end_dict
    if M is None:
      for node in H.nodes():
        start_dict[node] = f_date
    if M is not None:
      for node in H.nodes():
        node_dict = L.nodes[node]
        if M.has_node(node):
          pass
        elif node_dict.get('start') is None:
          start_dict[node] = f_date
        else:
          pass
      
    nx.set_node_attributes(L, start_dict, "start")

    M = H

    edge_predict = 0
    for u, v, p in preferential_edges[-100:]:
      if (H.has_edge(u,v)):
        edge_predict += 1
    jacc_predict = 0
    for u, v, p in jacc_edges[-100:]:
      if (H.has_edge(u,v)):
        jacc_predict += 1
    
    # change over time
    density = nx.density(H)
    print("Network density", f_date, density)
    clustering = average_clustering(H)
    preferential_predict = edge_predict/100
    jacc_edges_predict = jacc_predict/100
    gender_assort(H)
    set_bridges(H)
    print("female bridges", female_bridges)
    if male_bridges > 0:
      bridge_pct = female_bridges/male_bridges
    else:
      bridge_pct = 0
    this_dict = {
        "year": f_date,
        "year_count": year_count,
        "congregation": congregation,
        "denomination": denomination,
        "density": density, 
        "clustering": clustering, 
        "preferential_edges": preferential_predict, 
        "jacc_edges": jacc_edges_predict,
        "gender_assort": gender_assortivity,
        "male_bridges": male_bridges,
        "female_bridges": female_bridges,
        "male": len([x for x,y in H.nodes(data=True) if y['gender']=="male"]),
        "female": len([x for x,y in H.nodes(data=True) if y['gender']=="female"]),
        "nodes": H.number_of_nodes(),
        "bridge_pct": bridge_pct,
        "network_pct": len([x for x,y in H.nodes(data=True) if y['gender']=="female"]) / len([x for x,y in H.nodes(data=True) if y['gender']=="male"])
        }
    listofrows.append(this_dict)

    # predict nodes to be created
    preds = nx.preferential_attachment(H)
    preferential_edges = sorted(preds, key=lambda x: x[-1])
    jacc = nx.jaccard_coefficient(H)
    jacc_edges = sorted(jacc, key=lambda x: x[-1])
    
    # nx.draw(H,node_size=20)
    # plt.show()
    start_date += 1
    year_count += 1

### Export density over time

In [ ]:
def export_density():
  print("export density")
  densitydf = pd.DataFrame(listofrows)
  densityname = congregation + '/' + congregation + '_density.csv'
  densitydf.to_csv(densityname)

## Find communities

In [ ]:
def get_communities():
  print("get communities")
  global mod_assortivity
  global listcommunities
  # Detect the community structure of the graph which maximises the modularity using the Louvain heuristices
  partition = community_louvain.best_partition(L,resolution=1)
  nx.set_node_attributes(L, partition, 'modularity')
  listcommunities = set(partition.values())
  # print("Number of communities:", len(listcommunities))
  mod_assortivity = nx.attribute_assortativity_coefficient(L, attribute='modularity')
  # print(mod_assortivity)

## Export network

In [ ]:
def export_network():
  print("export network")
  data1 = json_graph.node_link_data(L)
  s1 = json.dumps(data1, indent=2, sort_keys=True)

  # print(s1)

  networkname = congregation + '/' + congregation + "_network.json"

  f = open(networkname, "w")
  f.write(s1)
  f.close()

## Export network statistics

In [ ]:
def sample_path_lengths(G, nodes=None, trials=1000):
    if nodes is None:
        nodes = list(G)
    else:
        nodes = list(nodes)

    pairs = np.random.choice(nodes, (trials, 2))
    lengths = [nx.shortest_path_length(G, *pair)
               for pair in pairs]
    return lengths

In [ ]:
def estimate_path_length(G, nodes=None, trials=1000):
    return np.mean(sample_path_lengths(G, nodes, trials))

In [ ]:
def gender_mix():
  print("get gender mixing")
  global mix
  global mixmat
  global mapping
  # measure of segregation
  mapping = {'male': 0, 'female': 1, 'unknown': 2}
  mixmat = nx.attribute_mixing_matrix(L, 'gender', mapping=mapping)
  mixmat[mapping['male'], mapping['female']]
  # count of attribute pairs
  mix = nx.attribute_mixing_dict(L, "gender")
  print("female-female", mix["female"]["female"])
  print("male-female", mix["male"]["female"])
  print("male-male", mix["male"]["male"])

In [ ]:
# int((nodegender.gender == 'female').sum())

### Summary statistics

In [ ]:
def get_degrees():
  print("get degree sequence")
  global average_degree
  degrees = dict(L.degree())
  sum_of_edges = sum(degrees.values())
  average_degree = sum_of_edges/L.number_of_nodes()

In [ ]:
def make_stats():
  print("make stats")
  global mapping
  start = df['year'].min()
  end = df['year'].max()
  yrs = end - start
  av_nodes = int(L.number_of_nodes()/yrs)
  av_edges = int(L.number_of_edges()/yrs)
  if male_bridges > 0:
    bridge_proportion = female_bridges/male_bridges
  else: 
    bridge_proportion = 0
  statsdict = {
      "congregation": congregation,
      "start": start,
      "end": end,
      "total nodes": G.number_of_nodes(),
      "total edges": G.number_of_edges(),
      "connected nodes": L.number_of_nodes(),
      "connected edges": L.number_of_edges(),
      "av_nodes": av_nodes,
      "av_edges": av_edges,
      "components": len(connected_subgraphs),
      "gender assortativity": gender_assortivity,
      "community assortativity": mod_assortivity,
      "degree assortativity": nx.degree_assortativity_coefficient(L),
      "community count": len(listcommunities),
      "average density": nx.density(L),
      "average degree": average_degree,
      "path length": estimate_path_length(L),
      "clustering": average_clustering(L),
      "female": int((nodegender.gender == 'female').sum()),
      "male": int((nodegender.gender == 'male').sum()),
      "unknown": int((nodegender.gender == 'unknown').sum()),
      "female_bridges": female_bridges,
      "male_bridges": male_bridges,
      "pct_bridges": (male_bridges + female_bridges) / L.number_of_nodes(),
      "gender_node_proportion": int((nodegender.gender == 'female').sum())/int((nodegender.gender == 'male').sum()),
      "bridge_proportion": bridge_proportion,
      "male-female": mixmat[mapping['male'], mapping['female']],
      "male-male": mixmat[mapping['male'], mapping['male']],
      "female-female": mixmat[mapping['female'], mapping['female']],
      "giant_percent_nodes": giant_percent_nodes,
      "giant_percent_edges": giant_percent_edges,
      "latitude": latitude,
      "longitude": longitude,
      "denomination": denomination
      # "triad_unconnected": len(triad_class[0]),
      # "triad_dyad": len(triad_class[1]),
      # "triad_incomplete": len(triad_class[2]),
      # "triad_complete": len(triad_class[3]),
  }
  print("make the dataframes")
  littlestatsdf = pd.DataFrame([statsdict])
  statsdfs.append(littlestatsdf)

  degseqdict = {
      "congregation": congregation,
      "degreeseq": degrees
  }
  littledegdf = pd.DataFrame([degseqdict])
  degdfs.append(littledegdf)
  # statsfile = congregation + '/' + congregation + "_network_stats.json"
  # with open(statsfile, "w") as outfile:
  #     json.dump(degseqdict, outfile, indent=2)

### Degree histogram

In [ ]:
def export_histogram(): 
  print("export histogram") 
  histdf = pd.DataFrame(nx.degree_histogram(G))
  histdf.rename(columns={0: 'count'}, inplace=True)
  histdf.index.names = ['degree']
  histdf
  histfile = congregation + '/' + congregation + "_hist.csv"
  histdf.to_csv(histfile)

## Download everything

In [ ]:
def download():
  shutil.make_archive(congregation, 'zip', congregation)
  # files.download(congregation + '.zip')

In [ ]:
def cleanup():
  shutil.rmtree(congregation)
  os.remove(congregation + '.zip')

In [ ]:
for c in remote_files:
  print(c['congregation'])
  setup(c['congregation'], c['filename'], c['denomination'], c['latitude'], c['longitude'])
  getedges()
  makegraph()
  make_giant()
  get_degseq()
  # get_triads()
  get_nodes()

  assign_gender()
  gender_unknown()
  set_gender()
  gender_assort(L)

  assign_race()
  set_race()

  set_bridges(L)

  set_degree()
  set_end()

  make_years()

  export_density()
  get_communities()

  export_network()

  gender_mix()
  get_degrees()

  make_stats()
  export_histogram()

  download()
  # cleanup()

statdf = pd.concat(statsdfs, ignore_index=True)
degdf = pd.concat(degdfs, ignore_index=True)
statdf.to_json("network_stats.json", orient='records', indent=2)
degdf.to_json("degreeseq.json", orient='records', indent=2)

gravesend-drc
Number of subgraphs: 69
Graph with 404 nodes and 673 edges
Graph with 215 nodes and 499 edges
Graph with 10 nodes and 10 edges
0.5321782178217822 0.7414561664190193
assign race
set race
set bridges
male bridges 23 female bridges 16
Graph with 10 nodes and 16 edges
Network density 1715 0.35555555555555557
set bridges
male bridges 0 female bridges 0
female bridges 0
Graph with 18 nodes and 30 edges
Network density 1716 0.19607843137254902
set bridges
male bridges 1 female bridges 0
female bridges 0
Graph with 33 nodes and 53 edges
Network density 1717 0.10037878787878787
set bridges
male bridges 2 female bridges 1
female bridges 1
Graph with 45 nodes and 81 edges
Network density 1718 0.08181818181818182
set bridges
male bridges 2 female bridges 1
female bridges 1
Graph with 45 nodes and 86 edges
Network density 1719 0.08686868686868687
set bridges
male bridges 2 female bridges 0
female bridges 0
Graph with 49 nodes and 92 edges
Network density 1720 0.0782312925170068
set br

In [ ]:
# for c in remote_files:
#   cleanup()